In [1]:
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import sent_tokenize
import pandas as pd
import re

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor

In [5]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/yunho/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [16]:
nltk.data.path.append("nltk_data")


def text_summarize(text):
    sentences = sent_tokenize(text) # 문장 단위로 분리
    vectorizer = TfidfVectorizer() # TF-IDF 벡터화
    sentence_vectors = vectorizer.fit_transform(sentences) # 문장 벡터화

    top_n = 1  # 문장 개수
    top_sentence_indices = sentence_vectors.sum(axis=1).argsort()[-top_n:] # 문장 벡터화 값이 큰 순서대로 정렬
    top_sentence_indices.sort() # 문장 벡터화 값이 큰 순서대로 정렬

    summary = " ".join(sentences[i.item()] for i in top_sentence_indices) # 문장 벡터화 값이 큰 순서대로 정렬된 문장들을 합침

    return summary


train = pd.read_csv("../Data/train.csv")
train["facts_summary"] = train["facts"].apply(lambda x: text_summarize(x))

test = pd.read_csv("../Data/test.csv")
test["facts_summary"] = test["facts"].apply(lambda x: text_summarize(x))

submit = pd.read_csv("../Data/sample_submission.csv")

# 부호 제거
def alpha_num(text):
    return re.sub(r"[^A-Za-z0-9 ]", "", text)


# 불용어 제거
stopwords = ["a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as",
             "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could",
             "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has",
             "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him",
             "himself",
             "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its",
             "itself",
             "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought",
             "our", "ours",
             "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some",
             "such", "than", "that",
             "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they",
             "they'd", "they'll",
             "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we",
             "we'd", "we'll",
             "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who",
             "who's", "whom",
             "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself",
             "yourselves"]


def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stopwords:
            final_text.append(i.strip())
    return " ".join(final_text)


# 중복제거
def delete_duplication(text):
    words = text.split()
    unique_words = []

    for word in words:
        if word not in unique_words:
            unique_words.append(word)

    result = " ".join(unique_words)
    return result


train["facts_summary"] = train["facts_summary"].str.lower()
test["facts_summary"] = test["facts_summary"].str.lower()
train["facts_summary"] = train["facts_summary"].apply(alpha_num).apply(remove_stopwords).apply(delete_duplication)
test["facts_summary"] = test["facts_summary"].apply(alpha_num).apply(remove_stopwords).apply(delete_duplication)

In [21]:
train["facts"][1234]

"Carol Anne Bond was found guilty of trying to poison her husband's mistress, Myrlinda Haynes, with toxic chemicals at least 24 times over the course of several months. A grand jury in the Eastern District of Pennsylvania charged Bond with two counts of possessing and using a chemical weapon, in violation of a criminal statute implementing the treaty obligations of the United States under the 1993 Chemical Weapons Convention. The grand jury also charged Bond with two counts of mail theft. Bond's attorneys argue that the statute was intended to deal with rogue states and terrorists and that their client should have been prosecuted under state law instead. Bond, a laboratory technician, stole the chemical potassium dichromate from the company where she worked. Haynes was not injured. Bond's husband had a child with Haynes while married to Bond. Haynes had contacted police and postal authorities after finding the chemicals at her home. In September 2009, the U.S. Court of Appeals for the 

In [22]:
train["facts_summary"][1234]

'carol anne bond found guilty trying poison husbands mistress myrlinda haynes toxic chemicals least 24 times course several months'